# <p style="padding: 15px; background-color: #778899; font-family: 'JetBrains Mono'; font-weight: bold; font-size: 150%; color: #f2f2f0; letter-spacing: 2px; text-align: center; border-radius: 8px;">Emotion Recognition Project</p>

In [ ]:
pip install torch==1.13.1 concrete-ml

In [1]:
import numpy as np
import pandas as pd
import os
import copy
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
data = pd.read_csv(
        "icml_face_data.csv"
    )

data

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...,...
35882,6,PrivateTest,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
35883,3,PrivateTest,178 174 172 173 181 188 191 194 196 199 200 20...
35884,0,PrivateTest,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
35885,3,PrivateTest,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1    Usage   35887 non-null  object
 2    pixels  35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [4]:
emotions = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral",
}
class_percentages = {}
num_labels = 7

In [5]:
!pip install torchvision==0.17.2

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [16]:
from torchvision import transforms
import numpy as np
import torch

def prepare_data(data, size=(12, 12)):
    """Prepare data for modeling.
    Input:
        data - DataFrame with labels and pixel data.
        size - New size of the images (tuple).
    Output:
        image and label tensors."""

    resize_transform = transforms.Resize(size)

    image_array = np.zeros(shape=(len(data), size[0], size[1]))
    image_label = np.array(list(map(int, data["emotion"])))

    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, " pixels"], dtype=int, sep=" ")
        image = np.reshape(image, (48, 48))
        image = torch.tensor(image).unsqueeze(0).unsqueeze(0).float()  # Convert to tensor, add batch and channel dimensions
        image = resize_transform(image)  # Apply resizing
        image_array[i] = image.squeeze().numpy()  # Remove dimensions and convert back to numpy

    image_array = torch.from_numpy(image_array)
    image_label = torch.from_numpy(image_label)

    return image_array.float(), image_label

# Example usage for 6x6 image size:
df_train_array, df_train_label = prepare_data(data[data[" Usage"] == "Training"], (6, 6))
df_val_array, df_val_label = prepare_data(data[data[" Usage"] == "PublicTest"], (6, 6))
df_test_array, df_test_label = prepare_data(data[data[" Usage"] == "PrivateTest"], (6, 6))

train_images = df_train_array.unsqueeze(1) / 255  # Add channel dimension and normalize
val_images = df_val_array.unsqueeze(1) / 255
test_images = df_test_array.unsqueeze(1) / 255

# <p style="padding: 15px; background-color: #778899; font-family: 'JetBrains Mono'; font-weight: bold; font-size: 125%; color: #f2f2f0; letter-spacing: 2px; text-align: center; border-radius: 8px;">Model Selection</p>

In [17]:
def to_categorical_torch(y, num_classes):
    return torch.eye(num_classes)[y]


def collate_fn(batch):
    """Collate function to process data batches."""
    inputs, targets = zip(*batch)
    return torch.stack(inputs), torch.stack(targets)


train_labels = to_categorical_torch(df_train_label, 7)
val_labels = to_categorical_torch(df_val_label, 7)
test_labels = to_categorical_torch(df_test_label, 7)

# Create TensorDatasets
train_dataset = TensorDataset(train_images, train_labels)
val_dataset = TensorDataset(val_images, val_labels)
test_dataset = TensorDataset(test_images, test_labels)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [31]:
from concrete.ml.torch.compile import compile_torch_model
import gc
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, stride=1, padding=0)  # Output size: 10x10
        self.conv2 = nn.Conv2d(8, 16, 3, stride=1, padding=0)  # Output size: 8x8
        self.pool = nn.MaxPool2d(2, 2)  # Output size: 4x4
        self.fc1 = nn.Linear( 4 * 4, n_classes)  # Flattened size: 16*4*4

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.flatten(1)
        x = self.fc1(x)
        return x



# Clear unused memory
gc.collect()
torch.cuda.empty_cache()

# Model instantiation and pruning
model = CNN(7)

try:
    q_module = compile_torch_model(model, train_images, rounding_threshold_bits=6, p_error=0.1)
    print("Model compilation successful")
except RuntimeError as e:
    print(f"Compilation failed: {e}")



Model compilation successful


### Device configuration


In [32]:
from tqdm import tqdm

def test_with_concrete(quantized_module, test_loader, use_sim):
    """Test a neural network that is quantized and compiled with Concrete ML."""
    all_y_pred = []

    fhe_mode = "simulate" if use_sim else "execute"

    for batch in tqdm(test_loader, desc="Testing"):
        data = batch
        if isinstance(batch, (list, tuple)) and len(batch) == 2:
            data = batch[0]  # Handles cases where data and targets are returned

        if isinstance(data, torch.Tensor):
            data = data.numpy()  # Ensure conversion to numpy for compatibility with FHE

        y_pred = quantized_module.forward(data, fhe=fhe_mode)
        y_pred = np.argmax(y_pred, axis=1)  # Assuming classification task
        all_y_pred.extend(y_pred)

    return np.array(all_y_pred)


In [33]:
q_module.fhe_circuit.keygen()

accuracy_test = test_with_concrete(
    q_module,
    test_loader,
    use_sim=True  # Ensure your model and system support running without simulation
)

accuracy_percentage = 100 * accuracy_test

Testing: 100%|██████████| 57/57 [00:41<00:00,  1.36it/s]


In [34]:
accuracy_scalar = accuracy_percentage[0]  # or accuracy_percentage[0]
print(f"Accuracy: {accuracy_scalar:.2f}%")

Accuracy: 100.00%
